In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# ===============================
# Notebook 1 - Custom CNN Models
# ===============================

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

# -----------------------
# Dataset Setup
# -----------------------
data_dir = "/content/drive/MyDrive/PBL3/data/chest_xray"
img_size = (224, 224)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, "train"),
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, "val"),
    image_size=img_size,
    batch_size=batch_size
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(data_dir, "test"),
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

class_names = train_ds.class_names

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(500).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# -----------------------
# Define CNN Models
# -----------------------
def build_baseline_cnn(input_shape=(224,224,3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')


    ])
    return model

def build_dropout_cnn(input_shape=(224,224,3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.25),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.25),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_batchnorm_cnn(input_shape=(224,224,3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

def build_dropout_batchnorm_cnn(input_shape=(224,224,3)):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.25),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Dropout(0.25),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# -----------------------
# Custom Callback for Logging Test Metrics at Specific Epochs
# -----------------------
class EpochLogger(tf.keras.callbacks.Callback):
    def __init__(self, model_name, test_ds, results, epochs_to_log=[10,20,30,40,50]):
        super().__init__()
        self.model_name = model_name
        self.test_ds = test_ds
        self.results = results
        self.epochs_to_log = epochs_to_log

    def on_epoch_end(self, epoch, logs=None):
        epoch_num = epoch + 1
        if epoch_num in self.epochs_to_log:
            y_true = np.concatenate([y.numpy() for x,y in self.test_ds], axis=0)
            y_prob = self.model.predict(self.test_ds, verbose=0)
            y_pred = (y_prob > 0.5).astype("int32").flatten()

            acc = accuracy_score(y_true, y_pred)
            prec = precision_score(y_true, y_pred, zero_division=0)
            rec = recall_score(y_true, y_pred, zero_division=0)
            f1 = f1_score(y_true, y_pred, zero_division=0)
            params = self.model.count_params()/1e6

            self.results.append({
                "Model": self.model_name,
                "Epoch": epoch_num,
                "Accuracy": acc,
                "F1-score": f1,
                "Precision": prec,
                "Recall": rec,
                "Params (M)": params
            })
            print(f"✅ Logged results at epoch {epoch_num}")

# -----------------------
# Training + Evaluation
# -----------------------
def train_and_evaluate(model, model_name, epochs=50):
    results = []
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    start = time.time()
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        verbose=1,
        callbacks=[EpochLogger(model_name, test_ds, results)]
    )
    end = time.time()
    train_time = (end-start)/60

    for r in results:
        r["Train Time (mins)"] = train_time

    return results

# -----------------------
# Run Experiments
# -----------------------
results = []

results.extend(train_and_evaluate(build_baseline_cnn(), "Baseline CNN"))
# results.extend(train_and_evaluate(build_dropout_cnn(), "Dropout CNN"))
# results.extend(train_and_evaluate(build_batchnorm_cnn(), "BatchNorm CNN"))
# results.extend(train_and_evaluate(build_dropout_batchnorm_cnn(), "Dropout+BatchNorm CNN"))

df = pd.DataFrame(results)
print(df)

df.to_csv("/content/custom_results_summary.csv", index=False)
print("✅ Saved to /content/custom_results_summary.csv")

Found 5226 files belonging to 2 classes.
Found 16 files belonging to 2 classes.
Found 634 files belonging to 2 classes.


NameError: name 'regularizers' is not defined